Step 1 - import required libraries 

In [1]:
#import required libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup #import the Beautiful soup functions to parse the data returned from the website

Step 2 - Using requests and BeautifulSoap libraries to scrap the data from wiki page and identified the table for data extract

In [2]:
# wiki URL to get Toronta neighborhood 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = BeautifulSoup(results.text,'html.parser')
all_tables=soup.find_all('table')
# assign the right table to object for data parsing
right_table = all_tables[0]

Step 3- Extract the table data from webpage and store in panda dataframe

In [3]:
# to get all rows and cols from this table
rows = right_table.find_all("tr")
rows_count = len(rows)
# define list to store the data 
postcode = []
borough = []
neighbourhood = []

for i in range(1,rows_count):
    cells = rows[i].findAll('td')
    postcode.append(cells[0].text)
    borough.append(cells[1].text)
    neighbourhood.append(cells[2].text.strip('\n'))

toronto_df = pd.DataFrame({'postcode':postcode,
                           'borough':borough,
                           'neighbourhood':neighbourhood})
toronto_df.head()

,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# print the shape of the dataframe
toronto_df.shape

(287, 3)

Step 4 - Clean dataframe by removing 'Not assigned' rows 

In [5]:
toronto_df_clean = toronto_df.copy()
toronto_df_clean.drop(toronto_df_clean[toronto_df_clean['borough'] =='Not assigned'].index,inplace=True)
toronto_df_clean.head()

,postcode,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
# to find out that Not assigned neighourhood. it will be same as borough
toronto_df_clean[toronto_df_clean['neighbourhood']=='Not assigned']

,postcode,borough,neighbourhood
7,M7A,Queen's Park,Not assigned


In [7]:
# to change Not assigned value in neighbourhood value to borough value
for index, row in toronto_df_clean.iterrows():
    if row['neighbourhood']=='Not assigned':
        toronto_df_clean.at[index,'neighbourhood']=toronto_df_clean.at[index,'borough']



In [8]:
# to check the values are updated in the dataframe
toronto_df_clean[toronto_df_clean['neighbourhood']==toronto_df_clean['borough']]

,postcode,borough,neighbourhood
7,M7A,Queen's Park,Queen's Park


Step 5 - Group the neighbourhood by Postcode and borough. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [11]:
toronto_df_grouped = toronto_df_clean.groupby(['postcode','borough']).agg(neighbourhood=('neighbourhood', ', '.join)).reset_index()
toronto_df_grouped.head()

,postcode,borough,neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Step 6 - print the shape of the final dataframe

In [12]:
toronto_df_grouped.shape

(103, 3)